<a href="https://www.kaggle.com/code/lucihere/fake-news-detection?scriptVersionId=231606816" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Contents
1) **Dependencies**
2) **Data Extraction**
3) **Data Wrangling**
4) **Machine Learning Models**
5) **Advanced Machine Learning Models**
6) **Results Analysis**

# 1) Dependencies
This section is for importing any relevent dependencies for this project.

In [ ]:
!pip install wandb
!pip install transformers

In [2]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb.login(key=user_secrets.get_secret("wandb_api"), relogin=True)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
import numpy as np
import pandas as pd
import os
from IPython.display import display
import chardet
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from glob import glob
import json
import string
import csv
import sys
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn import tree
from wandb.sklearn import plot_precision_recall, plot_feature_importances, plot_confusion_matrix
from wandb.sklearn import plot_class_proportions, plot_learning_curve, plot_roc
from wordcloud import WordCloud
import matplotlib.pyplot as plt, seaborn as sb
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import pickle
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from xgboost import XGBClassifier
from transformers import BertModel
import torch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, LSTM, Dense
import time

# 2) Data Extraction
I used Kaggle's Input directory to simplify the process of accumulating data into one single directory. <br>
I went through all the datasets individually and included them all in `/kaggle/inputs` so that I can parse through them all at once.
## 2.1) CSV file retrieval
The PHEME dataset is the only dataset that has events/data points split into seperate subdirectories, so we will deal with it later.<br>
For this section, even with a lot of subdirectories, metadata and other files to go through individually, a brief overview of the data shows the existence of many text files, CSV and TSV files in every dataset. So I'll create a script to go through all of the directories and just retrieve all them.<br>
<!-- <br> -->
<!-- 
**Following is the list of all English datasets from the list provided** (with file names in `/kaggle/inputs` for reference)**, marking the ones used:**
- [ ] [WELFake dataset](https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification) **(fake-news-classification)**
- [ ] [ISOT Fake News Dataset](https://www.kaggle.com/datasets/emineyetm/fake-news-detection-datasets) **(fake-news-detection-datasets)**
- [ ] [FakeNewsNet created by ASU](https://www.kaggle.com/datasets/mdepak/fakenewsnet) **(fakenewsnet)**
- [ ] [The LIAR Benchmark dataset](https://www.kaggle.com/datasets/lucihere/liar-dataset) **(liar-dataset)**
- [x] [PHEME dataset for Rumour Detection and Veracity Classification](https://www.kaggle.com/datasets/usharengaraju/pheme-dataset?select=PHEME_veracity) **(pheme-data)**
- [ ] [Truth Seeker Dataset 2023](https://www.kaggle.com/datasets/paulinepeps/truth-seeker-dataset-2023-truthseeker2023) **(truth-seeker-dataset-2023-truthseeker2023)**
- [ ] ["Fake News", another unnamed dataset from Kaggle](https://www.kaggle.com/datasets/hassanamin/textdb3/data) **(textdb3)**
- [x] ["TwitterFakeNews", a GitHub repository of scraped Twitter data](https://github.com/s-helm/twitterfakenews) **(twitterfakenews)**
- [ ] ["Fake and Real News Data", an unnamed dataset from Kaggle](https://www.kaggle.com/datasets/subhajournal/fake-and-real-news-data) **(fake-and-real-news-data)**
- [ ] [Fighting an Infodemic: COVID-19 Fake News Dataset](https://arxiv.org/ftp/arxiv/papers/2011/2011.03327.pdf) **()** -->

In [ ]:
# Lists to store DataFrames and data files
text_files = []
csv_dataframes = []
tsv_dataframes = []

# Lists to store the corresponding paths and filenames for reference
bad_pathnames = []
txt_pathnames = []
csv_pathnames = []
tsv_pathnames = []

bad_filenames = []
txt_filenames = []
csv_filenames = []
tsv_filenames = []

count_txt = 0
count_csv = 0
count_tsv = 0

for dirname, _, filenames_in_dir in os.walk('/kaggle/input'):
    for filename in filenames_in_dir:
        # Check if the file is a txt file
        if filename.endswith('.txt'):
            file_path = os.path.join(dirname, filename)
            print(f"Found Text file {count_txt}: {file_path}")
            count_txt += 1
            try:
                with open(file_path) as f:
                    data = f.read()
                    text_files.append(data)
                    txt_pathnames.append(file_path)
                    txt_filenames.append(filename)
            except Exception as e:
                print(f"  - Error loading {file_path}: {str(e)}")
                print("-" * 80)
                bad_pathnames.append(file_path)
                bad_filenames.append(filename)
            pass
        
        # Check if the file is a CSV file
        if filename.endswith('.csv'):
            file_path = os.path.join(dirname, filename)
            print(f"Found CSV file {count_csv}: {file_path}")
            count_csv += 1
            try:
                df = pd.read_csv(file_path)
                csv_dataframes.append(df)
                csv_pathnames.append(file_path)
                csv_filenames.append(filename)

            except Exception as e:
                try: 
                    with open(file_path, 'rb') as f:
                        data = f.read()
                    encoding = chardet.detect(data)["encoding"]
                    df = pd.read_csv(file_path, sep='delimiter', encoding=encoding, header=None, on_bad_lines='warn')
                    csv_dataframes.append(df)
                    csv_pathnames.append(file_path)
                    csv_filenames.append(filename)
            
                except Exception as e:
                    print(f"  - Error loading {file_path}: {str(e)}")
                    print("-" * 80)
                    bad_pathnames.append(file_path)
                    bad_filenames.append(filename)
            pass
            
        #Check if a file is a TSV file
        if filename.endswith('.tsv'):
            file_path = os.path.join(dirname, filename)
            print(f"Found TSV file {count_tsv}: {file_path}")
            count_tsv += 1
            try:
                df = pd.read_csv(file_path, sep='\t')
                tsv_dataframes.append(df)
                tsv_pathnames.append(file_path)
                tsv_filenames.append(filename)
                
            except Exception as e:
                try: 
                    with open(file_path, 'rb') as f:
                        data = f.read()
                    encoding = chardet.detect(data)["encoding"]
                    df = pd.read_csv(file_path, sep='\t', encoding=encoding)
                    tsv_dataframes.append(df)
                    tsv_pathnames.append(file_path)
                    tsv_filenames.append(filename)
                
                except Exception as e:
                    print(f"  - Error loading {file_path}: {str(e)}")
                    print("-" * 80)
                    bad_pathnames.append(file_path)
                    bad_filenames.append(filename)
            pass

#### **Run the cell below to get a glimpse of each dataframe and which file it points to.**

In [ ]:
for i in range(len(dataframes)):
    print(f"\n{i}) {filenames[i]}")
    print(f"Filepath: {pathnames[i]}")
    print(f"Number of rows:{dataframes[i].shape}\n")
    # display(dataframes[i].head(1))

#### **NOTE 2.1.1)**
The above output currently doesnt resemble the number of inputs used.
#### **NOTE 2.1.2)**
Looking through the text files, I could find a lot of unnecessary data that wont be useful to us in any way, so Ill skip them altogether. Feel free to verify by running `print(text_files[0])` in a new cell. You can choose any of the indices apart from 0, I dont recommend printing everything, since there is a huge chunk of data and it takes a lot of time to even print.

## 2.2) PHEME Dataset Retrieval
The following code is for individually going through all the sub-directories in the PHEME Veracity Dataset.

In [8]:
jsons = glob("/kaggle/input/pheme-data/**/*.json", recursive=True)
annotation_path = ""
source_tweets_path = ""

In [51]:
pheme_data = []
for i in jsons:
    #This skips over hidden binary files
    if i.startswith("."):
        continue

    elif "non-rumours/" in i:
        source_tweets_path = i
        if os.path.exists(source_tweets_path):
            with open(source_tweets_path, "r", encoding="utf-8") as f:
                tweet_data = json.load(f)
                if "text" in tweet_data:
                    label = 0
                    pheme_data.append({"text": tweet_data["text"], "label": label})
                else:
                    continue
                    
    elif "rumours/" in i and not "non-rumours" in i:
        source_tweets_path = i
        if os.path.exists(source_tweets_path):
            with open(source_tweets_path, "r", encoding="utf-8") as f:
                tweet_data = json.load(f)
                if "text" in tweet_data:
                    label = 1
                    pheme_data.append({"text": tweet_data["text"], "label": label})
                else:
                    continue
                
    # elif "/source-tweets" in i:
    #     source_tweets_path = i
    #     if os.path.exists(source_tweets_path):
    #         with open(source_tweets_path, "r", encoding="utf-8") as f:
    #             tweet_data = json.load(f)

    # elif i.endswith("annotation.json"):
    #     annotation_path = i

    #     if os.path.exists(annotation_path):
    #         with open(annotation_path, "r", encoding="utf-8") as f:
    #             annotation = json.load(f)
    #             if annotation.get("is_rumour")=="rumour": # Assign label from the annotation JSON
    #                 label = 1
    #             elif annotation.get("is_rumour")=="nonrumour":
    #                 label = 0
    else:
        label = pd.NA
        pheme_data.append({"text": pd.NA, "label": label})

#### **NOTE 2.2.1)** 
The above code only extracted the JSON files which are processable and have relevant annotations. There are a lot of sub directories that I believe the dataset was partly corrupted.<br><br>
<img src="attachment:47774470-dd02-4570-8bec-f7e16b34b16b.png" width=1000px><br>

**Fig. 1) Based on this official dataset's description, there shouldve be more tweets from 9 different events, but 7 among them were corrupted.** <br><br><br>

<img src="attachment:66a70f59-cfb6-43d0-88fb-0136e9056288.png" width=1500px><br>

**Fig. 2) One of the corrupted tweet files shown on NeoVIM.** <br><br><br>

<img src="attachment:cfcc70a7-2d34-4997-863b-927ce2b6cbbb.png" width=2000px><br>

**Fig. 3) How the non-corrupted tweet JSON should look like.** <br>

In [92]:
pheme_df = pd.DataFrame(pheme_data)
pheme_df.dropna(axis=0, inplace=True)
pheme_df.rename(columns={"content":"text"}, inplace=True)
pheme_count_fake = (pheme_df["label"] == 1).sum()
pheme_count_true = (pheme_df["label"] == 0).sum()
pheme_count_na = (pheme_df["label"] == pd.NA).sum()
print(f"Number of true tweets in the PHEME Dataset: {pheme_count_true}")
print(f"Number of fake tweets in the PHEME Dataset: {pheme_count_fake}")
print(f"Number of unclassified tweets in the PHEME Dataset: {pheme_count_na}")

,text,label
0,@tariqnasheed show his face and have Mike Brow...,1
1,@tariqnasheed anonymous named the wrong guy? D...,1
2,@tariqnasheed did you see how he was deliverin...,1
3,@tariqnasheed maybe it's an alias to cover his...,1
4,@tariqnasheed What do you think the outcome is...,1
...,...,...
111234,Gurlitt Kunstmuseum Bern update: There is no ...,0
111235,On the eve of a major announcement regarding t...,0
111236,"Swiss museum, German officials to announce fat...",0
111237,#Gurlitt's overlooked heirs want control of hi...,0


In [127]:
current_df = pd.read_csv("/kaggle/input/output-for-twitterfakesnews-csv/output_for_twitterfakenews.csv")
current_df = current_df.loc[:, ['text', 'label']]
current_df.dropna(axis=0, inplace=True)
curr_df = pd.concat([current_df[current_df['label']=='0'], current_df[current_df['label']=='1']])
curr_df['label'] = curr_df['label'].astype('int64')
output_csv = pd.concat([curr_df, pheme_df], ignore_index=True)
output_csv.dropna(inplace=True)
outp = pd.read_csv("/kaggle/working/output_for_pheme_and_twitter.csv", engine='python')
outp.dropna(axis=0, inplace=True)
outp['label'] = outp['label'].astype('int64')
outp

,text,label
0,washington reuters the head of a conservative...,0
1,washington reuters transgender people will be...,0
2,washington reuters the special counsel invest...,0
3,washington reuters trump campaign adviser geo...,0
4,seattlewashington reuters president donald tr...,0
...,...,...
156146,Gurlitt Kunstmuseum Bern update: There is no ...,0
156147,On the eve of a major announcement regarding t...,0
156148,"Swiss museum, German officials to announce fat...",0
156149,#Gurlitt's overlooked heirs want control of hi...,0


## 2.3) Combining the other datasets

In [145]:
truthseek_df = dataframes[12]
truthseek_df = truthseek_df.loc[:, ['tweet','BinaryNumTarget']]
truthseek_df.rename(columns = {"tweet": "text", "BinaryNumTarget": "label"}, inplace=True)
truthseek_df['label'] = truthseek_df['label'].astype('int64')
truthseek_df.dropna(axis=0, inplace=True)
output_csv2 = pd.concat([outp, truthseek_df], ignore_index = True)
output_csv2.to_csv("output_pheme_truthseeker_twitter.csv", index=False, header=True)

,text,label
0,washington reuters the head of a conservative...,0
1,washington reuters transgender people will be...,0
2,washington reuters the special counsel invest...,0
3,washington reuters trump campaign adviser geo...,0
4,seattlewashington reuters president donald tr...,0
...,...,...
290303,Joe Biden's family owned African slaves....\n\...,0
290304,"Joe Bidens great, great grandfather was a slav...",0
290305,"@ChevyChaseToGo ""Joe Bidens great-grandfather ...",0
290306,@JoeBiden Facts are Bidens VP Kamala Harris Gr...,0


In [171]:
liar_df = dataframes[14].iloc[:,[2,1]]
liar_df.columns = ['text', 'label']
liar_df1 = dataframes[15].iloc[:,[2,1]]
liar_df1.columns = ['text', 'label']
liar_df2 = dataframes[16].iloc[:,[2,1]]
liar_df2.columns = ['text', 'label']
full_liar_df = pd.concat([liar_df, liar_df1, liar_df2], ignore_index=True)
liar_mapping = {'true': 0, 'false': 1, 'half-true': 1, 'pants-fire': 1, 'barely-true': 1, 'mostly-true': 0}
full_liar_df['label'] = full_liar_df['label'].map(liar_mapping)
main_datasets_df = pd.concat([output_csv2, full_liar_df], ignore_index = True)

In [ ]:
reverse_mapping = {1:0,0:1}
welfake_df = dataframes[8].loc[:, ['text','label']]
welfake_df['label'] = welfake_df['label'].map(reverse_mapping)
main_datasets_df = pd.concat([main_datasets_df, welfake_df], ignore_index=True)
main_datasets_df.dropna(axis=0, inplace=True)
#csv.field_size_limit(sys.maxsize)
final_dataframe = pd.read_csv("/kaggle/input/final-news-dataset/cleaned_main_datasets.csv", engine='python')

In [ ]:
isot_fake_df = pd.read_csv("/kaggle/input/fake-news-detection-datasets/News _dataset/Fake.csv")
isot_fake_df = isot_fake_df['text']
isot_fake_df = pd.concat([isot_fake_df, pd.DataFrame([1] * len(isot_fake_df))], axis=1, names=["text","label"], ignore_index=True)
isot_fake_df.columns = ["text", "label"]
isot_real_df = pd.read_csv("/kaggle/input/fake-news-detection-datasets/News _dataset/True.csv")
isot_real_df = isot_real_df['text']
isot_real_df = pd.concat([isot_real_df, pd.DataFrame([0] * len(isot_real_df))], axis=1, names=["text","label"], ignore_index=True)
isot_real_df.columns = ["text", "label"]
final_dataframe = pd.concat([final_dataframe, isot_real_df, isot_fake_df], ignore_index=True)
final_dataframe

In [49]:
db3_mapping = {"FAKE": 1, "TRUE": 0}
textdb3_df = pd.read_csv("/kaggle/input/textdb3/fake_or_real_news.csv")
db3_df = textdb3_df.loc[:,["text","label"]]
db3_df['label'] = db3_df['label'].map(db3_mapping)
db3_df.fillna(value=0, inplace=True)
final_dataframe = pd.concat([final_dataframe, db3_df], ignore_index = True)
textdb3_df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [64]:
fakenewsnet_df = pd.read_csv("/kaggle/input/fake-and-real-news-data/Fake_Real_News_Data.csv")
fakenewsnet_df = fakenewsnet_df.loc[:,["text","label"]]
fakenewsnet_df["label"] = fakenewsnet_df["label"].map(db3_mapping)
fakenewsnet_df.fillna(value=0, inplace=True)
final_dataframe = pd.concat([final_dataframe, fakenewsnet_df], ignore_index = True)
final_dataframe.dropna(axis=0, inplace=True)
final_dataframe

,text,label
0,washington reuters the head of a conservative...,0.0
1,washington reuters transgender people will be...,0.0
2,washington reuters the special counsel invest...,0.0
3,washington reuters trump campaign adviser geo...,0.0
4,seattlewashington reuters president donald tr...,0.0
...,...,...
432747,Obama To Limit Police Acquisition Of Some Mili...,0.0
432748,BNI Store Oct 29 2016 EU using taxpayer money ...,1.0
432749,Next Story → Judge Judy LOSES IT on Hood Rat: ...,1.0
432750,America’s Streets Will Run With Blood- Mike Ad...,1.0


In [77]:
last_fake_df_1 = pd.read_csv("/kaggle/input/fakenewsnet/BuzzFeed_fake_news_content.csv")
last_fake_df_2 = pd.read_csv("/kaggle/input/fakenewsnet/PolitiFact_fake_news_content.csv")
last_real_df_1 = pd.read_csv("/kaggle/input/fakenewsnet/BuzzFeed_real_news_content.csv")
last_real_df_2 = pd.read_csv("/kaggle/input/fakenewsnet/PolitiFact_real_news_content.csv")
last_fake_df_1 = pd.concat([last_fake_df_1["text"], pd.DataFrame([1] * len(last_fake_df_1["text"]))], axis=1, names=["text","label"], ignore_index=True)
last_fake_df_1.columns = ["text", "label"]
last_fake_df_2 = pd.concat([last_fake_df_2["text"], pd.DataFrame([1] * len(last_fake_df_2["text"]))], axis=1, names=["text","label"], ignore_index=True)
last_fake_df_2.columns = ["text", "label"]
last_real_df_1 = pd.concat([last_real_df_1["text"], pd.DataFrame([0] * len(last_real_df_1["text"]))], axis=1, names=["text","label"], ignore_index=True)
last_real_df_1.columns = ["text", "label"]
last_real_df_2 = pd.concat([last_real_df_2["text"], pd.DataFrame([0] * len(last_real_df_2["text"]))], axis=1, names=["text","label"], ignore_index=True)
last_real_df_2.columns = ["text", "label"]
final_dataframe = pd.concat([final_dataframe, last_fake_df_1, last_fake_df_2, last_real_df_1, last_real_df_2], ignore_index=True)
final_dataframe.dropna(axis=0, inplace=True)
final_dataframe

,text,label
0,washington reuters the head of a conservative...,0.0
1,washington reuters transgender people will be...,0.0
2,washington reuters the special counsel invest...,0.0
3,washington reuters trump campaign adviser geo...,0.0
4,seattlewashington reuters president donald tr...,0.0
...,...,...
433169,"KALLSTADT, Germany — Few places in Germany are...",0.0
433170,Hollywood loses yet another one of their deare...,0.0
433171,"As my 25th wedding anniversary approached, I t...",0.0
433172,Story highlights Trump was sitting in a chair ...,0.0


In [75]:
final_dataframe.to_csv("main_datasets.csv",index=False, header=True)

# 3) Data Wrangling
Now that I have queried all the data files and created one big data set. We shall wrangle the data into something easier to process.<br><br>
- [x] [FakeNewsNet created by ASU](https://www.kaggle.com/datasets/mdepak/fakenewsnet) **(fakenewsnet)**
- [x] [WELFake dataset](https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification) **(fake-news-classification)**
- [x] ["TwitterFakeNews", a GitHub repository of scraped Twitter data](https://github.com/s-helm/twitterfakenews) **(twitterfakenews)**
- [x] ["Fake News", another unnamed dataset from Kaggle](https://www.kaggle.com/datasets/hassanamin/textdb3/data) **(textdb3)**
- [x] [ISOT Fake News Dataset](https://www.kaggle.com/datasets/emineyetm/fake-news-detection-datasets) **(fake-news-detection-datasets)**
- [x] ["Fake and Real News Data", an unnamed dataset from Kaggle](https://www.kaggle.com/datasets/subhajournal/fake-and-real-news-data) **(fake-and-real-news-data)**
- [x] [Truth Seeker Dataset 2023](https://www.kaggle.com/datasets/paulinepeps/truth-seeker-dataset-2023-truthseeker2023) **(truth-seeker-dataset-2023-truthseeker2023)**
- [x] [The LIAR Benchmark dataset](https://www.kaggle.com/datasets/lucihere/liar-dataset) **(liar-dataset)**
- [x] [PHEME dataset for Rumour Detection and Veracity Classification](https://www.kaggle.com/datasets/usharengaraju/pheme-dataset?select=PHEME_veracity) **(pheme-data)**

In [4]:
final_dataframe = pd.read_csv("/kaggle/input/cleaned-main-datasets-csv/cleaned_main_datasets.csv")
final_dataframe

,text,label
0,washington reuters the head of a conservative...,0.0
1,washington reuters transgender people will be...,0.0
2,washington reuters the special counsel invest...,0.0
3,washington reuters trump campaign adviser geo...,0.0
4,seattlewashington reuters president donald tr...,0.0
...,...,...
433358,kallstadt germany — few places in germany are ...,0.0
433359,hollywood loses yet another one of their deare...,0.0
433360,as my th wedding anniversary approached i trie...,0.0
433361,story highlights trump was sitting in a chair ...,0.0


Going through an example of the text, there seems to be a lot of unnecessary symbols in the data. We can use Regex expressions to remove them.<br>

In [44]:
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\[.*?\]", "", text)
    text = re.sub(r"\\W","", text)
    text = re.sub(r"\n", "", text)
    text = re.sub("https?:://\S+|www\.\S+","", text)
    text = re.sub("<.*?>+", "", text)
    text = re.sub("\w*\d\w", "", text)
    return text
    
def labeler(series, label):
    new_column = {label: [label] * len(series)}
    new_column = pd.DataFrame(new_column)
    output = pd.concat([series, new_column], ignore_index = False)
    output.columns = ['text', 'label']
    output['label'].fillna(label)
    return output

In [5]:
final_dataframe['text'] = final_dataframe['text'].apply(clean_text)
final_dataframe.dropna(axis=0, inplace=True)
final_dataframe['label'].value_counts() # 0 is True news and 1 is Fake news

label
0.0    235919
1.0    197255
Name: count, dtype: int64

In [11]:
vectorizer = TfidfVectorizer(max_features = 500, stop_words='english', max_df=0.7)
X, Y = vectorizer.fit_transform(final_dataframe['text']).toarray(), final_dataframe['label']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=69)

In [17]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
pd.DataFrame(X_train).to_csv("x_train.csv", index=False)
pd.DataFrame(X_test).to_csv("x_test.csv", index=False)

pd.DataFrame(y_train).to_csv("y_train.csv", index=False)
pd.DataFrame(y_test).to_csv("y_test.csv", index=False)

In [4]:
X_train = pd.read_csv("/kaggle/input/cleaned-main-datasets-csv/cleaned_split_dataset/x_train.csv")
X_test = pd.read_csv("/kaggle/input/cleaned-main-datasets-csv/cleaned_split_dataset/x_test.csv")
y_train = pd.read_csv("/kaggle/input/cleaned-main-datasets-csv/cleaned_split_dataset/y_train.csv")
y_test = pd.read_csv("/kaggle/input/cleaned-main-datasets-csv/cleaned_split_dataset/y_test.csv")

# 4) Model Building
- [x] **Logistic Regression** 
- [ ] **Support Vector Machine (SVM)**
- [ ] **Decision Trees**
- [ ] **XGBoost**
- [ ] **Passive Aggressive Classifier**
- [ ] **BERT**
- [ ] **CNN-RNN Hybrid**


In [5]:
test_size = 0.2
labels = [0,1]
named_labels = ["Real News", "Fake News"]
input_shape = (500, 1) # CNN-RNN Hybrid
project_name = 'fake-news-detection-models'

In [13]:
names = ["Logistic_Regressor",
         "Support_Vector_Machine",
        "Decision_Tree",
         "XGBoost",
         "Passive_Aggressive_Classifier"]

complex_names = ["BERT",
         "CNN_RNN_hybrid"]

classifiers = [
    LogisticRegression(), 
    svm.SVC(kernel="linear", C=0.025, random_state=42),
    tree.DecisionTreeClassifier(max_depth=5, random_state=42),
    XGBClassifier(n_estimators=2, max_depth=2, learning_rate=0.05, objective='binary:logistic'),
    PassiveAggressiveClassifier(),
]

complex_classifiers = [
    BertModel.from_pretrained("bert-base-uncased", torch_dtype=torch.float16, attn_implementation="sdpa"),
    Sequential([
        Input(shape=input_shape),
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(128, return_sequences=False),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
]

In [14]:
def simple_wandb_run(model_index, names, classifiers, X_train, X_test, y_train, y_test, named_labels, project_name):
    start_time = time.time()
    
    model = classifiers[model_index]
    name = names[model_index]
    X_train = X_train.values
    X_test = X_test.values
    y_train = y_train.values.ravel(order='C')
    y_test = y_test.values.ravel(order='C')
    data_loaded_time = time.time()
    print(f"Data loaded in {data_loaded_time - start_time}s..")
    
    model.fit(X_train, y_train)
    model_fitting_time = time.time()
    print(f"Model fitted in {model_fitting_time - data_loaded_time}s..")
    
    model_params = model.get_params()
    filename = name + '.sav'
    pickle.dump(model, open(filename, 'wb'))
    model_saving_time = time.time()
    print(f"Model saved at /kaggle/working/{filename} in {model_saving_time - model_fitting_time}s..")
    
    y_pred = model.predict(X_test)
    y_probas = model.predict_proba(X_test)
    model_predictions_time = time.time()
    print(f"Predictions completed in {model_predictions_time - model_saving_time}s..")

    wandb.init(project=project_name, name=name, config = model_params)
    wandb_init_time = time.time()
    print(f"Weights and Biases Run initialized in {wandb_init_time - model_predictions_time}s..")
    
    wandb.config.update({"test_size" : test_size,
                    "train_len" : len(X_train),
                    "test_len" : len(X_test)})
    
    plot_class_proportions(y_train, y_test, named_labels)
    plot_learning_curve(model, X_train, y_train)
    plot_roc(y_test, y_probas, named_labels)
    plot_precision_recall(y_test, y_probas, named_labels)
    plot_feature_importances(model)
    plot_confusion_matrix(y_test, y_pred, named_labels)
    wandb_plots_time = time.time()
    print(f"Plots completed in {wandb_plots_time - wandb_init_time}s!")
    
    wandb.finish()
    wandb_finish_time = time.time()
    print(f"Run closed in {wandb_finish_time - wandb_init_time}!")

## 4.1) Logistic Regression

In [15]:
simple_wandb_run(0, names, classifiers, X_train, X_test, y_train, y_test, named_labels, project_name)

Data loaded in 0.00010156631469726562s..
Model fitted in 4.225067138671875s..
Model saved at /kaggle/working/Logistic_Regressor.sav in 0.0019168853759765625s..
Predictions completed in 0.16667866706848145s..
Weights and Biases Run initialized in 0.0170135498046875s..


wandb: WARNING wandb uses only 10000 data points to create the plots.


Plots completed in 105.29735255241394s!


Run closed in 107.70126581192017!


## 4.2) Support Vector Machine

In [ ]:
simple_wandb_run(1, names, classifiers, X_train, X_test, y_train, y_test, named_labels, project_name)

Data loaded in 8.296966552734375e-05s..


## 4.3) Decision Tree

In [ ]:
simple_wandb_run(2, names, classifiers, X_train, X_test, y_train, y_test, named_labels, project_name)

## 4.4) XGBoost

In [ ]:
simple_wandb_run(3, names, classifiers, X_train, X_test, y_train, y_test, named_labels, project_name)

In [1]:
simple_wandb_run(4, names, classifiers, X_train, X_test, y_train, y_test, named_labels, project_name)

NameError: name 'simple_wandb_run' is not defined